# 16. Deep Q-Networks (DQN)  
**Author**: Your Name  
**Date**: June 9, 2025  

## Introduction

DQN combines **Q-Learning** with **deep neural networks** to handle environments with **large or continuous state spaces**.

- **Type**: Model-Free, Off-Policy, Value-Based RL  
- **Task**: Policy learning via function approximation  
- **Goal**: Approximate Q(s, a) using a neural network


In [ ]:
# Suppress TensorFlow logging and disable GPU if not needed
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'            # Suppress INFO and WARNING logs
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'           # Prevent GPU detection to speed up CPU-only workflows

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from collections import deque

# Environment and simulation
import gymnasium as gym

# Machine learning (TensorFlow/Keras)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Plotting configuration
sns.set_style('whitegrid')


In [ ]:
env = gym.make("CartPole-v1")
state_space_size = env.observation_space.shape[0]
action_space_size = env.action_space.n

print("State Space Size:", state_space_size)
print("Action Space Size:", action_space_size)


In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        self.policy_network = self._build_model()
        self.target_network = self._build_model()
        self.update_target_network()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_network(self):
        self.target_network.set_weights(self.policy_network.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.policy_network.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward if done else reward + self.gamma * np.amax(self.target_network.predict(next_state, verbose=0)[0])
            target_f = self.policy_network.predict(state, verbose=0)
            target_f[0][action] = target
            self.policy_network.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

agent = DQNAgent(state_space_size, action_space_size)


In [ ]:
total_episodes = 500
batch_size = 32
update_target_every = 10
scores = []

print("--- Starting Training ---")

for e in range(total_episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_space_size])

    for time in range(500):
        action = agent.act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_space_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
            scores.append(time + 1)
            print(f"Episode {e+1}/{total_episodes}, Score: {time+1}, Epsilon: {agent.epsilon:.2f}")
            break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

    if e % update_target_every == 0:
        agent.update_target_network()

print("--- Training Finished ---")
env.close()


In [ ]:
moving_avg_scores = pd.Series(scores).rolling(100).mean()

plt.figure(figsize=(12, 6))
plt.plot(scores, alpha=0.5, label='Episode Score')
plt.plot(moving_avg_scores, linewidth=2, label='100-Episode Moving Average')
plt.title('DQN Training Performance on CartPole-v1')
plt.xlabel('Episodes')
plt.ylabel('Score (Time Steps)')
plt.legend()
plt.grid(True)
plt.show()


## Key Takeaways

- DQNs use neural networks to generalise Q-values over continuous/high-dimensional states.
- Experience Replay and Fixed Q-Targets are critical for training stability.
- A foundational algorithm behind many advanced RL methods.
- Still limited in continuous **action** spaces (policy gradient methods required).

## Further Reading

- DeepMind (2015): *Human-level control through deep reinforcement learning*
- Sutton & Barto: *Reinforcement Learning: An Introduction*
- [TensorFlow DQN Tutorial](https://www.tensorflow.org/tutorials/reinforcement_learning/intro)
- [StatQuest: Deep Q-Learning Explained](https://www.youtube.com/watch?v=79pmNdyxEGo)
